<a href="https://colab.research.google.com/github/OptimalDecisions/sports-analytics-foundations/blob/main/sa-getting-started/Pandas_Example_Merging_Datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pandas Example of Shaping data by Merging Dataframes


<img src = "../img/sa_logo.png" width="100" align="left">

Ram Narasimhan

<br><br><br>





We have two csv files. One of them contains tons of NBA games, one row per game.
However, it only has a TEAM_ID for both the home and away teams. It is much easier (for a human!) to work with NBA team names and cities.

In this example, we are going to merge (left join) two times.

STEP 1: The first merge will bring in all the HOME TEAM data into the data frame.

Step 2: We will merge again, this time bringing in all the AWAY TEAM's details.

STEP 3: After dropping some redundant columns, we will write it to a CSV file for future use.

In [66]:

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
%matplotlib inline


In [67]:

teams = pd.read_csv('/nba_teams.csv')
games = pd.read_csv('/games.csv')

In [68]:
games.shape, teams.shape

((26651, 21), (30, 14))

In [69]:
games.columns, teams.columns

(Index(['GAME_DATE_EST', 'GAME_ID', 'GAME_STATUS_TEXT', 'HOME_TEAM_ID',
        'VISITOR_TEAM_ID', 'SEASON', 'TEAM_ID_home', 'PTS_home', 'FG_PCT_home',
        'FT_PCT_home', 'FG3_PCT_home', 'AST_home', 'REB_home', 'TEAM_ID_away',
        'PTS_away', 'FG_PCT_away', 'FT_PCT_away', 'FG3_PCT_away', 'AST_away',
        'REB_away', 'HOME_TEAM_WINS'],
       dtype='object'),
 Index(['LEAGUE_ID', 'TEAM_ID', 'MIN_YEAR', 'MAX_YEAR', 'ABBREVIATION',
        'NICKNAME', 'YEARFOUNDED', 'CITY', 'ARENA', 'ARENACAPACITY', 'OWNER',
        'GENERALMANAGER', 'HEADCOACH', 'DLEAGUEAFFILIATION'],
       dtype='object'))

In [70]:
useful_team_cols = ['TEAM_ID', 'ABBREVIATION', 'NICKNAME', 'CITY']

In [71]:
teams = teams[useful_team_cols]

## STEP 1

### Merge (left join) Games with the `teams` df, matching on home team ID

In [72]:
# Merge based on TEAM_ID_home

merged_games_home = pd.merge(games, teams,
                             left_on='TEAM_ID_home',
                             right_on='TEAM_ID',
                             how='left')



In [73]:
merged_games_home.head()

,GAME_DATE_EST,GAME_ID,GAME_STATUS_TEXT,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,...,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS,TEAM_ID,ABBREVIATION,NICKNAME,CITY
0,2022-12-22,22200477,Final,1610612740,1610612759,2022,1610612740,126.0,0.484,0.926,...,0.478,0.815,0.321,23.0,44.0,1,1610612740,NOP,Pelicans,New Orleans
1,2022-12-22,22200478,Final,1610612762,1610612764,2022,1610612762,120.0,0.488,0.952,...,0.561,0.765,0.333,20.0,37.0,1,1610612762,UTA,Jazz,Utah
2,2022-12-21,22200466,Final,1610612739,1610612749,2022,1610612739,114.0,0.482,0.786,...,0.470,0.682,0.433,20.0,46.0,1,1610612739,CLE,Cavaliers,Cleveland
3,2022-12-21,22200467,Final,1610612755,1610612765,2022,1610612755,113.0,0.441,0.909,...,0.392,0.735,0.261,15.0,46.0,1,1610612755,PHI,76ers,Philadelphia
4,2022-12-21,22200468,Final,1610612737,1610612741,2022,1610612737,108.0,0.429,1.000,...,0.500,0.773,0.292,20.0,47.0,0,1610612737,ATL,Hawks,Atlanta


We've added 4 columns with the first merge. Let's make sure that the resulting dataframe does indeed reflect that.

In [74]:

merged_games_home.shape

(26651, 25)

## STEP 2
### Merge again with team, but this time by matching `TEAM_ID_away` with the `TEAM_ID` in `teams`

When doing repeated merges, it is possible but the same column shows up multiple times. To avoid this confusion, we add `suffixes`. In this case `_h` for home team and `_a` for the away team.


In [ ]:
merged_games_all = pd.merge(merged_games_home, teams,
                             left_on='TEAM_ID_away',
                             right_on='TEAM_ID', how='left',
                             suffixes=('_h', '_a'))



The warning can be ignored this time. It is occurring because we already had a column called TEAM_ID_home, and the `merge` produces yet another column (based on the suffix) called TEAM_ID_home. The same df cannot have 2 columns of the same name.

In [76]:
merged_games_all.shape, merged_games_all.columns


((26651, 29),
 Index(['GAME_DATE_EST', 'GAME_ID', 'GAME_STATUS_TEXT', 'HOME_TEAM_ID',
        'VISITOR_TEAM_ID', 'SEASON', 'TEAM_ID_home', 'PTS_home', 'FG_PCT_home',
        'FT_PCT_home', 'FG3_PCT_home', 'AST_home', 'REB_home', 'TEAM_ID_away',
        'PTS_away', 'FG_PCT_away', 'FT_PCT_away', 'FG3_PCT_away', 'AST_away',
        'REB_away', 'HOME_TEAM_WINS', 'TEAM_ID_h', 'ABBREVIATION_h',
        'NICKNAME_h', 'CITY_h', 'TEAM_ID_a', 'ABBREVIATION_a', 'NICKNAME_a',
        'CITY_a'],
       dtype='object'))

In [77]:
# Drop redundant TEAM ID columns
merged_games_all.drop(['TEAM_ID_h', 'TEAM_ID_a'], axis=1, inplace=True)


In [78]:
merged_games_all.sample()

,GAME_DATE_EST,GAME_ID,GAME_STATUS_TEXT,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,...,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS,ABBREVIATION_h,NICKNAME_h,CITY_h,ABBREVIATION_a,NICKNAME_a,CITY_a
18609,2004-01-17,20300570,Final,1610612740,1610612753,2003,1610612740,91.0,0.442,0.625,...,0.313,20.0,38.0,1,NOP,Pelicans,New Orleans,ORL,Magic,Orlando


In [79]:
merged_games_all.columns

Index(['GAME_DATE_EST', 'GAME_ID', 'GAME_STATUS_TEXT', 'HOME_TEAM_ID',
       'VISITOR_TEAM_ID', 'SEASON', 'TEAM_ID_home', 'PTS_home', 'FG_PCT_home',
       'FT_PCT_home', 'FG3_PCT_home', 'AST_home', 'REB_home', 'TEAM_ID_away',
       'PTS_away', 'FG_PCT_away', 'FT_PCT_away', 'FG3_PCT_away', 'AST_away',
       'REB_away', 'HOME_TEAM_WINS', 'ABBREVIATION_h', 'NICKNAME_h', 'CITY_h',
       'ABBREVIATION_a', 'NICKNAME_a', 'CITY_a'],
      dtype='object')

In [80]:
merged_games_all.shape

(26651, 27)

Everything looks good! Let save it to a csv file for future use.

## Step 3
### Write the dataframe to a new csv file

In [81]:
# prompt: write merged_games_all to a csv, without the integer index

merged_games_all.to_csv('nba_games_with_names.csv', index=False)
